# MRS Voxel to ROI

get the MRS voxel coordinates data from a CSV of RDAs headers, make a sphere in subject space centered on the voxel center and coregister it in MNI space

In [5]:
import csv
import json
import os
import numpy as np
import nibabel as nib
import pandas as pd
import suspect

# Make a DataFrame with .rda headers' data

In [6]:
from suspect import MRSData, transformation_matrix

import numpy
import struct
import re

# The RDA format consists of a large number of key value pairs followed by raw
# data. The values need to be cast into different datatypes depending on the
# key, this dictionary stores a mapping of key to datatype.

rda_types = {
    "floats": ["PatientWeight", "TR", "TE", "TM", "TI", "DwellTime", "NumberOfAverages",
               "MRFrequency", "MagneticFieldStrength", "FlipAngle", "SliceThickness",
               "FoVHeight", "FoVWidth", "PercentOfRectFoV", "PixelSpacingRow",
               "PixelSpacingCol", "VOIPositionSag", "VOIPositionCor",
               "VOIPositionTra", "VOIThickness", "VOIPhaseFOV", "VOIReadoutFOV",
               "VOIReadoutVOV", "VOINormalSag", "VOINormalCor", "VOINormalTra",
               "VOIRotationInPlane", "FoV3D", "PixelSpacing3D"],
    "integers": ["SeriesNumber", "InstanceNumber", "AcquisitionNumber", "NumOfPhaseEncodingSteps",
                 "NumberOfRows", "NumberOfColumns", "VectorSize", "EchoNumber",
                 "NumberOf3DParts", "HammingFilterWidth", "NumberOfEchoes"],
    "strings": ["PatientID", "PatientName", "StudyDescription", "PatientBirthDate",
                "StudyDate", "StudyTime", "PatientAge", "SeriesDate", "SeriesTime",
                "SeriesDescription", "ProtocolName", "PatientPosition", "ModelName",
                "StationName", "InstitutionName", "DeviceSerialNumber", "InstanceDate",
                "InstanceTime", "InstanceComments", "SequenceName", "SequenceDescription",
                "Nucleus", "TransmitCoil", "PatientSex", "HammingFilter", "FrequencyCorrection"],
    "float_arrays": ["PositionVector", "RowVector", "ColumnVector"],
    "integer_arrays": ["CSIMatrixSize", "CSIMatrixSizeOfScan", "CSIGridShift"],
    "string_arrays": ["SoftwareVersion"],
    "dictionaries": ["TransmitRefAmplitude"]
}


def load_rda(json_el):
    global rda_types
    header_dict = {}
    for i in json_el:
        key = i
        value = json_el[i]
        try:
            if key in rda_types["strings"]:
                header_dict[key] = value
            elif key in rda_types["integers"]:
                header_dict[key] = int(value)
            elif key in rda_types["floats"]:
                header_dict[key] = float(value)
            elif "[" in key and "]" in key:
                # could be a dict or a list
                key, index = re.split("\]|\[", key)[0:2]
                if key in rda_types["dictionaries"]:
                    if key not in header_dict:
                        header_dict[key] = {}
                    header_dict[key][index] = value
                else:
                    # not a dictionary, must be a list
                    if key in rda_types["float_arrays"]:
                        value = float(value)
                    elif key in rda_types["integer_arrays"]:
                        value = int(value)
                    index = int(index)
                    # make sure there is a list in the header_dict, with enough entries
                    if not key in header_dict:
                        header_dict[key] = []
                    while len(header_dict[key]) <= index:
                        header_dict[key].append(0)
                    header_dict[key][index] = value
        except:
            header_dict[key] = value
        # now we can read the data
        complex_data = []


    # some .rda files have a misnamed field, correct this here
    if "VOIReadoutFOV" not in header_dict:
        if "VOIReadoutVOV" in header_dict:
            header_dict["VOIReadoutFOV"] = header_dict.pop("VOIReadoutVOV")

    # combine positional elements in the header
    voi_size = (header_dict["VOIReadoutFOV"],
                header_dict["VOIPhaseFOV"],
                header_dict["VOIThickness"])
    voi_center = (header_dict["VOIPositionSag"],
                  header_dict["VOIPositionCor"],
                  header_dict["VOIPositionTra"])
    voxel_size = (header_dict["PixelSpacingCol"],
                  header_dict["PixelSpacingRow"],
                  header_dict["PixelSpacing3D"])

    x_vector = numpy.array(header_dict["RowVector"])
    y_vector = numpy.array(header_dict["ColumnVector"])

    to_scanner = transformation_matrix(x_vector, y_vector, numpy.array(voi_center), voxel_size)

    # put useful components from the header in the metadata
    metadata = {
        "voi_size": voi_size,
        "position": voi_center,
        "voxel_size": voxel_size,
        "protocol": header_dict["ProtocolName"],
        "to_scanner": to_scanner,
        "from_scanner": numpy.linalg.inv(to_scanner)
    }

    return MRSData(complex_data,
                   header_dict["DwellTime"] * 1e-6,
                   header_dict["MRFrequency"],
                   te=header_dict["TE"],
                   transform=to_scanner,
                   metadata=metadata)


In [7]:
csv_path = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET2/MRS_Voxel_masks/rda_headers_NM2_2.csv'

In [8]:
import pandas as pd

df = pd.read_csv(csv_path, sep=',', header=0).drop(columns='Unnamed: 0')


In [9]:
jsn = json.loads(df.to_json(orient="records"))

# 2 Make a Nifty with the MRS Voxel as 10mm radius sphere

In [10]:
# Credits NLTools: https://github.com/cosanlab/nltools
def sphere(nii_path, r, p):
    """ create a sphere of given radius at some point p in the brain mask
    Args:
        r: radius of the sphere
        p: point (in coordinates of the brain mask) of the center of the sphere
    """
    nii = nib.load(nii_path)
    dims = nii.shape

    x, y, z = np.ogrid[-p[0]:dims[0]-p[0], -p[1]:dims[1]-p[1], -p[2]:dims[2]-p[2]]
    mask = x*x + y*y + z*z <= r*r

    activation = np.zeros(dims)
    #print(activation.sum())
    activation[mask] = 1
    #print(activation.sum())
    #activation = np.multiply(activation, nii.get_fdata())
    activation = activation + nii.get_fdata()
    #print(activation.sum())
    activation = nib.Nifti1Image(activation, nii.affine, nii.header)

    # return the 3D numpy matrix of zeros containing the sphere as a region of ones
    return activation

In [11]:
# Sphere radius in mm
sphere_radius = 10

In [12]:
# Mask for T1w filename
t1_file_mask = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET2/Structural_analysis_fs7/{sub_id}/{sub_id}.mUNIbrain_DENskull_SPMmasked.nii.gz'

In [13]:
# target directories
subject_space_sphere_path = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET2/MRS_Voxel_masks/Subjekt_space_spheres'
subject_space_voxel_path = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET2/MRS_Voxel_masks/Subject_space_Voxels'

In [14]:
# create target directories if don't exist
for d in [subject_space_sphere_path, subject_space_voxel_path]:
    if not os.path.isdir(d):
        os.mkdir(d)

In [17]:
ids = set([i for i in df['PatientID'] if '106' in i or '107' in i or '108' in i])
print(ids)

{'NeuroMET2-108-T1', 'NeuroMET2-106-T1', 'NeuroMET2-107-T1'}


In [18]:
for j in range(len(df)):
    try:
        i=json.loads(df.iloc[j].to_json())
        if i['PatientID'] in ids:     
            #print(n)
            sub_id = i['PatientID'].replace('-','_')
            print(sub_id)
            t1_path = t1_file_mask.format(sub_id=sub_id)
            #print(t1_path)
            rda = load_rda(i)
            base_name = t1_path.split('/')[-1][:-7] 
            #print(base_name)
            # get voxel coordinates for the center of mrs voxel
            t1 = suspect.image.load_nifti(t1_path)
            mrs_centre = rda.to_scanner(0, 0, 0)
            #print(mrs_centre)
            mrs_centre_index = t1.from_scanner(*mrs_centre).round().astype(int)
            #print(mrs_centre_index)
            # zero all voxels and put 1 for mrs voxel center
            img = nib.load(t1_path)
            new_data = img.get_fdata().copy()
            new_data[:,:,:] = 0
            new_data[mrs_centre_index[0],mrs_centre_index[1],mrs_centre_index[2]] = 1
            #create and save nifti
            new_img = nib.Nifti1Image(new_data, img.affine, img.header)
            nib.save(new_img, os.path.join(subject_space_voxel_path, base_name + '_mrs_voxel_center.nii.gz'))
            #make a sphere and save it
            sssphere = sphere(os.path.join(subject_space_voxel_path, base_name + '_mrs_voxel_center.nii.gz'),
                         r=sphere_radius,
                         p=mrs_centre_index)
            nib.save(sssphere, os.path.join(subject_space_sphere_path, base_name + '_mrs_voxel_sphere.nii.gz'))
            ids.remove(i['PatientID'])
            print('done')
    except:
        print('problem')

NeuroMET2_106_T1
done
NeuroMET2_107_T1
done
NeuroMET2_108_T1
done


In [19]:
# Coregister in MNI space

In [20]:
import ants
import os

In [32]:
# ANTs fixed image
mni_std = '/usr/share/fsl/data/standard/MNI152_T1_0.5mm.nii.gz'
ants_mni_std = ants.image_read(mni_std)

In [33]:
# ANTs moving image
mov_mask = str(os.path.join(subject_space_sphere_path, '{sub_id}.mUNIbrain_DENskull_SPMmasked_mrs_voxel_sphere.nii.gz'))

In [40]:
# mask for ANTs transform filenames
transform1_mask = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET2/MNI_coreg/{sub_id}/{sub_id}_MNIcoreg0GenericAffine.mat'
transform2_mask = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET2/MNI_coreg/{sub_id}/{sub_id}_MNIcoreg1Warp.nii.gz'

In [41]:
mni_sp_path='/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET2/MRS_Voxel_masks/MNI_space_spheres'
#Create directory for MNI152 coregistered masks if it doesn't exists
if not os.path.isdir(mni_sp_path):
    os.mkdir(mni_sp_path)

In [42]:
dest_path_mask = os.path.join(mni_sp_path,'{}_MRS_Voxel_MNIwarped.nii.gz')

In [43]:
ids = set([i for i in df['PatientID'] if '106' in i or '107' in i or '108' in i])
print(ids)

{'NeuroMET2-108-T1', 'NeuroMET2-106-T1', 'NeuroMET2-107-T1'}


In [44]:
for sub_id in ids:
    #try:
    sub_id=sub_id.replace('-','_')
    print(sub_id)
    transform1 = transform1_mask.format(sub_id=sub_id)
    transform2 = transform2_mask.format(sub_id=sub_id)
    moving = mov_mask.format(sub_id=sub_id)
    ants_mov = ants.image_read(moving)
    res = ants.apply_transforms(fixed=ants_mni_std,
                     moving=ants_mov,
                     transformlist = [transform2, transform1])
    ants.image_write(res, dest_path_mask.format(sub_id))
    print('Done')
    #except:
    #    pass

NeuroMET2_108_T1
Done
NeuroMET2_106_T1
Done
NeuroMET2_107_T1
Done
